MOABB to NY format

.CSV to NY

this code convert the data sets in .CSV format to NY format which it can be easily read in Julia and Python. 

The .npz file holds the data and the stimulation vector while the .yml file holds the metadata. 

It has been specifically conceived for BCI data.

This script is for Grosswentrup2009


In [1]:
#import essential libraries

import numpy as np
import pandas as pd
import glob, os, sys, yaml
from yaml import CLoader as Loader, CDumper as Dumper
import moabb
from moabb.paradigms import MotorImagery


In [3]:
# Verify the epochs shape, labels, and metadata of BI2013a if wanted
# Epoch shape >>> trials X channels X time_samples

paradigm = MotorImagery()
dataset = moabb.datasets.GrosseWentrup2009()
subjects = dataset.subject_list

epochs, labels, meta = paradigm.get_data(dataset=dataset, subjects = [subjects[0]], return_epochs = True)


Choosing from all possible events


Used Annotations descriptions: ['left_hand', 'right_hand']
Adding metadata with 3 columns


c:\Users\doumif\.conda\envs\py310\lib\site-packages\moabb\datasets\preprocessing.py:278: UserWarning: warnEpochs <Epochs | 300 events (all good), 0 – 7 s (baseline off), ~1.00 GiB, data loaded,
 'right_hand': 150
 'left_hand': 150>
  warn(f"warnEpochs {epochs}")
c:\Users\doumif\.conda\envs\py310\lib\site-packages\moabb\paradigms\base.py:350: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  X = mne.concatenate_epochs(X)


Adding metadata with 3 columns
300 matching events found
No baseline correction applied


In [ ]:
# Extracting all the information we need to create the .yml file for metadata
paradigm = MotorImagery()
dataset = moabb.datasets.Zhou2016()

label_dict = dataset.event_id
subjects = dataset.subject_list
data_d = dataset.get_data()

for sub_id in list(data_d.keys()):
    epochs, _, _ = paradigm.get_data(dataset=dataset, subjects = [sub_id], return_epochs=True)

    for session_id in list(data_d[sub_id].keys()):
        session_eeg = np.array([]).reshape(len(epochs.ch_names),0)
        session_stim = np.array([]).reshape(0,)

        for run_id in list(data_d[sub_id][session_id].keys()):
            session_stim = np.concatenate((session_stim, data_d[sub_id][session_id][run_id].get_data()[-1,:]), axis=0)
            session_eeg = np.concatenate((session_eeg, data_d[sub_id][session_id][run_id].pick_types(eeg=True).get_data()), axis=1)

In [4]:
dataset.event_id.keys()

dict_keys(['right_hand', 'left_hand'])

In [5]:
epochs.ch_names

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128']

In [9]:
# Forward your file with all .csv of the dataset

file_dir = "C:/Users/doumif/work/Prog/GrosseWentrup2009"
all_files = [os.path.join(file_dir, file) for file in os.listdir(file_dir)]
all_files #check if you have .csv files

['C:/Users/doumif/work/Prog/GrosseWentrup2009\\subject_01_session_01.csv',
 'C:/Users/doumif/work/Prog/GrosseWentrup2009\\subject_02_session_01.csv',
 'C:/Users/doumif/work/Prog/GrosseWentrup2009\\subject_03_session_01.csv',
 'C:/Users/doumif/work/Prog/GrosseWentrup2009\\subject_04_session_01.csv',
 'C:/Users/doumif/work/Prog/GrosseWentrup2009\\subject_05_session_01.csv',
 'C:/Users/doumif/work/Prog/GrosseWentrup2009\\subject_06_session_01.csv',
 'C:/Users/doumif/work/Prog/GrosseWentrup2009\\subject_07_session_01.csv',
 'C:/Users/doumif/work/Prog/GrosseWentrup2009\\subject_08_session_01.csv',
 'C:/Users/doumif/work/Prog/GrosseWentrup2009\\subject_09_session_01.csv',
 'C:/Users/doumif/work/Prog/GrosseWentrup2009\\subject_10_session_01.csv']

In [6]:
#Calc windowlength
samplingrate = 250 # original data was downsampled to 250Hz (original: 500Hz available in dataset description) 
windowlength = np.diff(dataset.interval)[0]*samplingrate
windowlength

1750

In [7]:
subject_list = list(range(1, 11))
subject_list

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [14]:
#YML creator 
#The Meta data of the Data set
#These variables and parameters should address once for each data set.  

#######################################Acquisition#######################################
filter= 'Low-pass 81 Hz, Butterworth 4th-order, forward-backward (a digital filter was applied because original data was downsampled)'
ground= 'N/A'
hardware= 'BrainAmp amplifiers'
reference= 'Cz'
samplingrate= 250 # original data was downsampled to 250Hz, original = 500Hz (available in dataset description)
sensors = epochs.ch_names
sensortype ='N/A'
software= 'MATLAB/Simulink'

######################################Documentation######################################
description= 'https://neurotechx.github.io/moabb/generated/moabb.datasets.GrosseWentrup2009.html#moabb.datasets.GrosseWentrup2009'
doi= 'https://doi.org/10.1109/TBME.2008.2009768'
investigators = "Moritz Grosse-Wentrup ; Christian Liefhold; Klaus Gramann; Martin Buss"
place = "Institute of Automatic Control Engineering (LSR), Munich, Germany; Max-Planck-Institute, Tubingen, Germany, Swartz Center for Computational Neuroscience, San Diego, USA"
repository= 'https://zenodo.org/records/1217449'
##id
condition= 'None'
database= 'GrosseWentrup2009'  
paradigm= 'MI'
timestamp= 2009

##########################################Stim###########################################
####Labels
event_labels = dict(left_hand= 1, right_hand = 2) 
nclasses= len(event_labels.keys())
offset = 0
windowlength = 1750 

#########################################Subjects########################################  
subjects=len(subject_list)
sessions = 1
runs = 1 

#function of YMLcreator
def YMLcreator():


  d = dict(acquisition=dict
          (filter= str(filter),
          ground=str(ground),
          hardware=str(hardware),
          reference=str(reference),
          samplingrate= samplingrate,
          sensors= sensors,
          sensortype=str(sensortype),
          software=str(software)
          )
          ,documentation=dict
          (description=str(description),
          doi=str(doi),
          investigators=str(investigators),
          place=str(place),
          repository=str(repository)
          )              
          ,
          formatversion=str('0.0.1')
          ,
          id=dict
          (condition=str(condition),
          database=str(database),
          paradigm=str(paradigm),
          subject=subject,
          session=session,
          run=run,
          timestamp=timestamp,
          )
          ,stim=dict
          (labels=event_labels,
          nclasses=nclasses,
          offset=offset,
          windowlength=windowlength,
          )
          
  )
  return d  

i=-1 #don't change i

for subject in range(1,subjects+1):
    for session in range(1,sessions+1):
        i=i+1
        filepath = all_files[i]
        for run in range(1,runs+1):
            d=YMLcreator()
            newpath=os.path.splitext(filepath)[0] + '.yml'
            with open(newpath, 'w') as file:
              documents = yaml.dump(d, file)

              file.write("""
##############################################################################
#                     GIPSA-lab standard for EEG time series (version 0.0.1) #
#                                Authors : Pedro Rodrigues and Marco Congedo #
#                                                        November 15th, 2019 #
##############################################################################

# This format has been conceived for easily sharing EEG data in Python and 
# Julia. Each file is understood as a separate recording. Data consist of two
# files. They have the same name and extensions `npz` and `yml` (this file).

# The `npz` file typically holds the EEG data matrix, a real matrix of 
# dimension num. of samples x num. of electrodes and a vector of integer with
# the tags for stimulations, with as many entries as number of samples. The 
# tags are 0 (zero) for no stimulation and then employs the natural numbers 
# (1, 2,...) for different stimulation classes.

# The `yml` file holds all meta-data info of the recording in `yml` format. 
# It holds two fields and four dictionaries:

# FIELDS:
#
# - paradigm: (string) the experimental paradigm, e.g., P300, MI, ... 
#             for Brain Computer Interfaces experiments
#
# - formatversion: (version) version of this metadata specification

# DICTIONARIES:
#
# - acquisition: (dictionary)
#
#   - filter: (string) filter setting of the EEG acquisition machine, 
#             specifying the type and specification. Ex: "Band-pass digital 
#             filter (0.01-70Hz)"
#   - ground: (string) location of the sensor used as ground. Ex: "Fpz"
#   - reference: (string) location of the sensor used as reference for the 
#                recording. Ex: "A1"
#   - hardware: (string) the commercial name and producer of the the EEG 
#               acquisition machine. Ex: "actiCHamp, Brain Products GmbH 
#               (Germany), DC amplifiers"
#   - software: (string) software used for acquiring and storing the data. 
#               Ex: OpenViBE, INRIA (France)
#   - samplingrate:(int) sampling rate. Ex: 128
#   - sensors: (array-like of strings) location of the sensors, excluding 
#              ground and reference.
#   - sensortype: (string) type, material and product name of electrodes. 
#                 Ex: Ag/AgCl, Braincap, Brain Products GmbH (Germany)
#
# - documentation: (dictionary)
#
#   - description: (string) link to a file or website describing the dataset
#   - doi: (string) digital object identifier of the dataset's documentation
#   - repository:  (string) link to the online repository where the data can 
#                  be downloaded
#
# - id: (dictionary)
#   
#   - database: (string) name of the database
#
# - stim: (dictionary)
#
#   - labels: (dictionary) dictionary with the labels and code of the 
#             stimulations
#       - nclasses: (int) number of classes for the stimulations
#       - offset: (int) offset, given in number of samples, with respect to 
#                 stimulation samples, defining the beginning of trials
#       - windowlength: (int) size of the window, given in number of sample, 
#                       defining the duration of trials
""")

In [13]:

##NPZ creator

#the Structure of the Data sets 
"""
The data should be in the form of  Samples*Channels, where channels may include a column for the time,
a column for the ground, a colum for the trigger, and a colum for the target. So, you need to drop the
columns of time and ground and add the two columns of trigger and target together(If you have not already done so).
"""

timestamp_col = 0  
stim_col= 129 # Stim col

#NPZ creator

def csv2npz(filepath):
    df = pd.read_csv(filepath, header=0)  # header = 0 to remove header
    df = np.array(df)

    # from V to µV
    raw_data = df[:, 1:-1] * 1e6  

    # Ensuite conversion en float32
    DATA = np.float32(raw_data)
    print(DATA.shape)

    # STIM col
    STIM = np.int16(df[:, stim_col]) 

    unique_values, counts = np.unique(STIM, return_counts=True)
    for value, count in zip(unique_values, counts):
        print(f"Dans {filepath} : Nombre de {value} = {count}")

    newpath = os.path.splitext(filepath)[0] + '.npz'
    np.savez(newpath, data=DATA, stim=STIM)

for f in all_files:
    csv2npz(f)

(779170, 128)
Dans C:/Users/doumif/work/Prog/GrosseWentrup2009\subject_01_session_01.csv : Nombre de 0 = 778870
Dans C:/Users/doumif/work/Prog/GrosseWentrup2009\subject_01_session_01.csv : Nombre de 1 = 150
Dans C:/Users/doumif/work/Prog/GrosseWentrup2009\subject_01_session_01.csv : Nombre de 2 = 150
(753590, 128)
Dans C:/Users/doumif/work/Prog/GrosseWentrup2009\subject_02_session_01.csv : Nombre de 0 = 753290
Dans C:/Users/doumif/work/Prog/GrosseWentrup2009\subject_02_session_01.csv : Nombre de 1 = 150
Dans C:/Users/doumif/work/Prog/GrosseWentrup2009\subject_02_session_01.csv : Nombre de 2 = 150
(758990, 128)
Dans C:/Users/doumif/work/Prog/GrosseWentrup2009\subject_03_session_01.csv : Nombre de 0 = 758690
Dans C:/Users/doumif/work/Prog/GrosseWentrup2009\subject_03_session_01.csv : Nombre de 1 = 150
Dans C:/Users/doumif/work/Prog/GrosseWentrup2009\subject_03_session_01.csv : Nombre de 2 = 150
(754050, 128)
Dans C:/Users/doumif/work/Prog/GrosseWentrup2009\subject_04_session_01.csv : Nom